In [1]:
import numpy as np
import slidingwindow as sw
import sys
import os

from mmdet.apis import init_detector, inference_detector
from mmcv.runner import load_checkpoint



# import shm_tools here
sys.path.append('..')
from shm_tools.shm_utils import imread, imwrite

In [2]:
# load checkpoint file and configuration 

config = './project_work_dirs/multiple_damage_detection/mask_rcnn_x101_64x4d_fpn_1x_bilin.py'
checkpoint = './project_work_dirs/multiple_damage_detection/epoch_48.pth'

device = 'cuda:0'
model = init_detector(config, checkpoint, device=device)


In [ ]:
# set the path of original images 
imsave_folder = '/media/uosshm/Backup Plus/시설물영상분석/신규학습자료/001. 교면/2강원본부/#area_with_spalling'

# run inference with the trained network and save images that contains detection result with confidence bigger than 0.5
for root, dirs, files in os.walk('/media/uosshm/Backup Plus/시설물영상분석/신규학습자료/001. 교면/2강원본부'):
    for file in files:
        if file.endswith(".JPG") or file.endswith(".jpg") :
#             print(os.path.join(root, file))
            img = imread(os.path.join(root, file))
            windows = sw.generate(img, sw.DimOrder.HeightWidthChannel, 1024*4, 0 )
            img_num = 0
        
            for window in windows: # Can a multithreading be used here?? 
                img_num += 1 

                img_subset = img[window.indices()]
                results = inference_detector(model, img_subset)
                bbox_results, seg_results = results
                for bbox in bbox_results[3]: 
                    if bbox[-1] > 0.2:
                        filename_base, _ = os.path.splitext(file)
                        img_num_str = str(img_num)
                        img_num_str = img_num_str.zfill(3)
                        imsave_filename = filename_base + '_' + img_num_str + '.jpeg'
                        imwrite(os.path.join(imsave_folder, imsave_filename), img_subset )
                        break
                
            
    